In [1]:
# Create iLO 5 environment variables
iLO5IP="ilo5"
iLO5URI="https://${iLO5IP}"

iLO5User="student"
iLO5Password='P@ssw0rd!'

# Minimum required Redfish headers
HeaderODataVersion="OData-Version: 4.0"
HeaderContentType="Content-Type: application/json"

# Data files
ResponseHeaders="ResponseHeaders.txt"   # Used to hold HTTP response headers
SessionData="./CreateSession-data.json" # Body/Workload used to create the Redfish session
cat > ${SessionData} << __EOF__
{
        "UserName": "$iLO5User",
        "Password": "$iLO5Password"
}
__EOF__

echo "Verifying that the remote iLO 5 is accessible on port 443 (HTTPS)"
nc -vz $iLO5IP 443 || echo -e "\nProblem: Ask for help in Slack"

Verifying that the remote iLO 5 is accessible on port 443 (HTTPS)
Connection to ilo5 443 port [tcp/https] succeeded!


In [2]:
echo "OData Supported operations:"
curl --insecure --silent --noproxy "localhost, 127.0.0.1"  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --request GET ${iLO5URI}/redfish/v1/ | jq '.ProtocolFeaturesSupported'

OData Supported operations:
{
  "ExpandQuery": {
    "ExpandAll": false,
    "Levels": true,
    "Links": false,
    "MaxLevels": 1,
    "NoLinks": true
  },
  "FilterQuery": true,
  "OnlyMemberQuery": true,
  "SelectQuery": false
}


In [ ]:
echo 'Create Session and print body response:' 

curl --dump-header  $ResponseHeaders \
     --insecure --noproxy "localhost, 127.0.0.1" --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" \
     --request POST --data "@$SessionData" \
     ${iLO5URI}/redfish/v1/SessionService/Sessions | jq

In [ ]:
Token=$(awk '/X-Auth-Token/ {print $NF}' $ResponseHeaders | tr -d '\r')
SessionLocation="$iLO5URI"$(awk '/^Loca.*Se/ {gsub("https://.*/red", "/red", $NF);print $NF}' $ResponseHeaders | tr -d '\r')

echo "Token: $Token"
echo -e "Session Location: $SessionLocation\n"

## Retrieve the Chassis collection URI

In [ ]:
ChassisCollectionURIs=$(curl --insecure --silent   \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $Token" \
  --request GET ${iLO5URI}/redfish/v1/Chassis | jq -r '.Members | .[] | ."@odata.id"')
  
echo $ChassisCollectionURIs


## For every chassis in the collection retrieve the Device

In [ ]:
DevicesURIs=""

for chassis in $ChassisCollectionURIs ; do
DU=$(curl --insecure --silent  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $Token" \
  --request GET ${iLO5URI}${chassis} | jq -r '[.Oem.Hpe.Links.Devices] | .[] | ."@odata.id"')
DevicesURIs="$DevicesURIs $DU" 
done 
 
echo $DevicesURIs

## For each Device URI, retrieve properties 

In [ ]:
echo "Selected device properties:"

for du in ${DevicesURIs}; do

curl --insecure --silent  \
  --header  "$HeaderContentType" --header "$HeaderODataVersion" \
  --header "X-Auth-Token: $Token" \
  --request GET ${iLO5URI}${du}'?$expand=.' | \
      jq '.Members | .[] | {Location: .Location, Name: .Name, ProductVersion: .ProductVersion, FirmwareVersion: .FirmwareVersion, Status: .Status }'
done

## Logout

In [ ]:
curl --insecure --silent \
     --header "$HeaderContentType" --header "$HeaderODataVersion" --header "X-Auth-Token: $BmcToken" \
     --request DELETE $SessionLocation | jq